# TEST Cnn_bar

In [1]:
import os
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import scipy.io as sio
import time

## Cnn_bar module

In [17]:
# Cnn_bar: to computing the barrier parameter
class Cnn_bar(nn.Module):
    """
    Predicts the barrier parameter.
    Attributes
    ----------
        conv2, conv3 (torch.nn.Conv1d): 1-D convolution layer
        lin          (torch.nn.Linear): fully connected layer
        avg       (torch.nn.AVgPool2d): average layer
        soft       (torch.nn.Softplus): Softplus activation function
    """
    def __init__(self):
        super(Cnn_bar, self).__init__()
        # in_channels: int,
        # out_channels: int, 
        # kernel_size: Union[T, Tuple[T]], 
        # stride: Union[T, Tuple[T]] = 1, 
        # padding: Union[T, Tuple[T]] = 0, 
        # dilation: Union[T, Tuple[T]] = 1,
        # groups: int = 1, 
        # bias: bool = True, padding_mode: str = 'zeros
        self.conv2  = nn.Conv1d(1, 1, 5,padding=2)
        self.conv3  = nn.Conv1d(1, 1, 5,padding=2)
        # What is the size of the output ?
        self.lin    = nn.Linear(16*1, 1)
        self.avg    = nn.AvgPool1d(4, 4)
        self.soft   = nn.Softplus()

    def forward(self, x):
        """
        Computes the barrier parameter.
        Parameters
        ----------
      	    x (torch.FloatTensor): images, size n*c*h*w 
        Returns
        -------
       	    (torch.FloatTensor): barrier parameter, size n*1*1*1
        """
        x = self.soft(self.avg(self.conv2(x)))
        x = self.soft(self.avg(self.conv3(x)))
        x = x.view(x.size(1), -1)
        x = self.soft(self.lin(x))
        x = x.view(x.size(0),-1,1,1)
        return x


## Tensor and test

In [18]:
batch_size = 1
# [ batch_size , nb_channel =1, signal]
x = np.arange(256).reshape(1,-1,256)
xt = torch.FloatTensor(x)
xt.shape

torch.Size([1, 1, 256])

In [20]:
cnn = Cnn_bar()

In [21]:
cnn

Cnn_bar(
  (conv2): Conv1d(1, 1, kernel_size=(5,), stride=(1,), padding=(2,))
  (conv3): Conv1d(1, 1, kernel_size=(5,), stride=(1,), padding=(2,))
  (lin): Linear(in_features=16, out_features=1, bias=True)
  (avg): AvgPool1d(kernel_size=(4,), stride=(4,), padding=(0,))
  (soft): Softplus(beta=1, threshold=20)
)

In [22]:
cnn(xt)

tensor([[[[0.6085]]]], grad_fn=<ViewBackward>)